<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">


# Web Scraping for Indeed.com and Predicting Salaries

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary predictor with Logistic Regression.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being able to extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use Logistic Regression or any other suitable classifier.

- Question: Why would we want this to be a classification problem?
- Answer: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10").

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

The URL here has many query parameters:

- `q` for the job search
- This is followed by "+20,000" to return results with salaries (or expected salaries >$20,000)
- `l` for a location 
- `start` for what result number to start on

In [1]:
base_url = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

In [2]:
#This is the new URL after a 301 redirect occurs.#
'https://www.indeed.com/jobs?q=data+scientist+$20,000&l=New+York&start=10'

'https://www.indeed.com/jobs?q=data+scientist+$20,000&l=New+York&start=10'

In [23]:
#Importing Dependencies
import requests
import bs4
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import csv


#Loading Selenium Dependencies
import selenium
from selenium import webdriver
import platform
import time
from random import randint

path = 'C:\Program Files (x86)\Google\Chrome\Application\chrome.exe'

if platform.system() == 'Windows':
    chromedriver = r'/Users/jamesanthonyphoenix/Desktop/GA/DSI9-project-submissions/James-Anthony-Phoenix/Project_4/chromedriver_win32.exe'
else:
    chromedriver = r'/Users/jamesanthonyphoenix/Desktop/GA/DSI9-project-submissions/James-Anthony-Phoenix/Project_4/chromedriver'
    

#Importing ML libraries

In [4]:
# How to operate a selenium web_browser.
browser = webdriver.Chrome(executable_path=chromedriver)
browser.get('https://python.org')
soup = BeautifulSoup(browser.page_source, 'html.parser')

KeyboardInterrupt: 

In [74]:
# How to operate a HTTP requests object.
req = requests.get(base_url)
soup = BeautifulSoup(req.text, 'html.parser')
results = []

for item in soup.find_all('div', class_='jobsearch-SerpJobCard'):
    results.append(item)

In [7]:
# How to operate a HTTP requests object.
req = requests.get('https://www.indeed.co.uk/jobs?q=data+science+£40,000&l=London')
soup = BeautifulSoup(req.text, 'html.parser')
results = []

for item in soup.find_all('div', class_='jobsearch-SerpJobCard'):
    if item.find('span', class_='salary') is not None:
        print('test')
    else:
        print('no salary')
#     results.append(item)


test
test
no salary
test
no salary
no salary
no salary
test
no salary
no salary
no salary
no salary
no salary
no salary
no salary
test
no salary
no salary
no salary


Let's look at one result more closely. A single `result` looks like

```
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=2480d203f7e97210&amp;jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a `nobr` element inside of a `td` element with `class='snip'`.
- The title of a job is in a link with class set to `jobtitle` and a `data-tn-element='jobTitle'`.  
- The location is set in a `span` with `class='location'`. 
- The company is set in a `span` with `class='company'`. 

### Write 4 functions to extract each item: location, company, job, and salary.

Example: 
```python
def extract_location_from_result(result):
    return result.find ...
```


- **Make sure these functions are robust and can handle cases where the data/field may not be available.**
    - Remember to check if a field is empty or `None` for attempting to call methods on it.
    - Remember to use `try/except` if you anticipate errors.
- **Test** the functions on the results above and simple examples.

In [26]:
def extract_salaries(soup, np=np):
    salaries = []
    for item in soup.find_all('div', class_='jobsearch-SerpJobCard'):
        try:
            salaries.append(item.find('span', class_='salary').getText().strip().replace('\n', ''))
        except:
            salaries.append(np.nan)
    return salaries

In [27]:
def extract_titles(soup, np=np):
    titles = []
    for item in soup.find_all('div', class_='jobsearch-SerpJobCard'):
        try:
            titles.append(item.find('div', class_='title').getText().strip())
        except:
            titles.append(np.nan)
    return titles

In [28]:
def extract_locations(soup, np=np):
    locations = []
    for item in soup.find_all('div', class_='jobsearch-SerpJobCard'):
        try:
            locations.append(item.find('span', class_='location').getText().replace('\n','').strip())
        except:
            locations.append(np.nan)
    return locations

In [29]:
def extract_companies(soup, np=np):
    companies = []
    for item in soup.find_all('div', class_='jobsearch-SerpJobCard'):
        try:
            companies.append(item.find('span', class_='company').getText().replace('\n','').strip())
        except:
            companies.append(np.nan)
    return companies

In [30]:
## 🔥🚀 #Let's scrape the companies ratings (the maximum width is 60px so we can use this as 100%) 🔥🚀

def extract_ratings(soup, np=np):
    import re
    ratings = []
    for item in soup.find_all('div', class_='jobsearch-SerpJobCard'):
        try:
            width = item.find('span',class_='rating').attrs['style']
            rating = float(findall(':([^p]+)', width)[0])
            ratings.append((round(((rating / 60) * 100))))
        except:
            ratings.append(np.nan)
    return ratings    

In [31]:
# Let's extract all of the delightful ahref <a> links for the individual job listings.

def extract_links(soup, np=np):
    urls = []
    for item in soup.find_all('div', class_='result'):
        try:
            urls.append('https://www.indeed.com' + item.find('a').attrs['href'])
        except:
            urls.append(np.nan)
    return urls

In [ ]:
#Testing Functions

-----------------------------------------------------------------------------------------------------------------------------

-----------------------------------------------------------------------------------------------------------------------------

In [16]:
def extract_salaries(soup, np=np):
    salaries = []
    for item in soup.find_all('div', class_='jobsearch-SerpJobCard'):
        if item.find('span', class_='salary') is not None:
            try:
                salaries.append(item.find('span', class_='salary').getText().strip().replace('\n', ''))
            except:
                salaries.append(np.nan)
        else:
            pass
    return salaries

In [17]:
def extract_titles(soup, np=np):
    titles = []
    for item in soup.find_all('div', class_='jobsearch-SerpJobCard'):
        if item.find('span', class_='salary') is not None:
            try:
                titles.append(item.find('div', class_='title').getText().strip())
            except:
                titles.append(np.nan)
        else:
            pass
    return titles

In [18]:
def extract_locations(soup, np=np):
    locations = []
    for item in soup.find_all('div', class_='jobsearch-SerpJobCard'):
        if item.find('span', class_='salary') is not None:
            try:
                locations.append(item.find('span', class_='location').getText().replace('\n','').strip())
            except:
                locations.append(np.nan)
        else:
            pass
    return locations

In [19]:
def extract_companies(soup, np=np):
    companies = []
    for item in soup.find_all('div', class_='jobsearch-SerpJobCard'):
        if item.find('span', class_='salary') is not None:
            try:
                companies.append(item.find('span', class_='company').getText().replace('\n','').strip())
            except:
                companies.append(np.nan)
        else:
            pass
    return companies

In [20]:
## 🔥🚀 #Let's scrape the companies ratings (the maximum width is 60px so we can use this as 100%) 🔥🚀

def extract_ratings(soup, np=np):
    import re
    ratings = []
    for item in soup.find_all('div', class_='jobsearch-SerpJobCard'):
        if item.find('span', class_='salary') is not None:
            try:
                width = item.find('span',class_='rating').attrs['style']
                rating = float(findall(':([^p]+)', width)[0])
                ratings.append((round(((rating / 60) * 100))))
            except:
                ratings.append(np.nan)
        else:
            pass
    return ratings    

In [21]:
# Let's extract all of the delightful ahref <a> links for the individual job listings.

def extract_links(soup, np=np):
    urls = []
    for item in soup.find_all('div', class_='result'):
        if item.find('span', class_='salary') is not None:
            try:
                urls.append('https://www.indeed.com' + item.find('a').attrs['href'])
            except:
                urls.append(np.nan)
        else:
            pass
    return urls

-----------------------------------------------------------------------------------------------------------------------------

Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).

#### Complete the following code to collect results from multiple cities and starting points. 
- Enter your city below to add it to the search.
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different.

In [11]:
from IPython.display import clear_output

In [12]:
YOUR_CITY = 'London'

In [13]:
city_list = ['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 
    'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
    'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', YOUR_CITY]

In [26]:
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 300 # Set this to a high-value (5000) to generate more results. 
# Crawling more results, will also take much longer. First test your code on a small number of results and then expand.

# results = {
#     'Salaries($)': [],
#     'Titles': [],
#     'Locations': [],
#     'Companies': [],
#     'Ratings': [],
#     'URL_Listings': []
# }


with open('test.csv', 'a') as f:
    w = csv.writer(f)
    for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 
        'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
        'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', YOUR_CITY]):

        page_index = 0

        for start in range(0, max_results_per_city):
            
            clear_output(wait=True)
            #Let's Create An Index For The URL's.
            page_index += 10

            #Let's Make The URL Dynamnic Via Formatting.
            user_agent = {'User-agent': 'Mozilla/5.0'}
            dynamic_url = url_template.format(city, page_index)
            req = requests.get(dynamic_url)
            soup = BeautifulSoup(req.text, 'html.parser')

#             Appending The Results To A Dictionary!
#             results['Salaries($)'].extend(extract_salaries(soup))
#             results['Titles'].extend(extract_titles(soup))
#             results['Locations'].extend(extract_locations(soup))
#             results['Companies'].extend(extract_companies(soup))
#             results['Ratings'].extend(extract_ratings(soup))
#             results['URL_Listings'].extend(extract_links(soup))
            
            
            #This method allows us to save intermediate results for web scraping via a CSVwriterows Append.
            data = [list(item) for item in list(zip(extract_salaries(soup),extract_titles(soup)
                                                    ,extract_locations(soup), extract_companies(soup),
                                                    extract_companies(soup), extract_ratings(soup), 
                                                    extract_links(soup)))]
            w.writerows(data)
            print(city, page_index)

KeyboardInterrupt: 

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [130]:
#To avoid Pandas truncating our Listing_URL columns, let's increase the max_colwidth.

pd.set_option("display.max_colwidth", 10000)
df = pd.DataFrame(results, columns = list(results.keys()))

AttributeError: 'list' object has no attribute 'keys'

Lastly, we need to clean up salary data. 

1. Only a small number of the scraped results have salary information - only these will be used for modeling.
1. Some of the salaries are not yearly but hourly or weekly, these will not be useful to us for now.
1. Some of the entries may be duplicated.
1. The salaries are given as text and usually with ranges.

#### Find the entries with annual salary entries, by filtering the entries without salaries or salaries that are not yearly (filter those that refer to hour or week). Also, remove duplicate entries.

In [61]:
#Let's filter out of all of the null salary entries and any entries that contain hour.

df = df[(df['Salaries($)'].notnull()) & (df['Salaries($)'].str.contains('hour') == False)]

NameError: name 'df' is not defined

#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary.

In [8]:
#Solving the string manipulation problem with Regex and an IF statement for months vs years.

def convert_digits(a):
    import re

    #If there is a month in the string.
    digits = []
    if 'month' in a:
        for x in a.split():
                x_ = x.replace(',','')
                x_ = re.sub('[$,£]', '', x_)
                if x_.isdigit():
                    year_figure = (int(x_) * 12)
                    digits.append(year_figure)

        if len(digits) == 1:
            return digits[0]

        else:
            return sum(digits) / len(digits)

    #If there is no month in the string.

    else:
        for x in a.split():
            x_ = x.replace(',','')
            x_ = re.sub('[$,£]', '', x_)
            if x_.isdigit():
                digits.append(int(x_))

        if len(digits) == 1:
            return digits[0]

        else:
            return sum(digits) / len(digits)

In [9]:
#Let's apply our formula to extract all of the rows.

df['Salaries($)'] = df['Salaries($)'].apply(convert_digits)

NameError: name 'df' is not defined

In [10]:
### Data Cleaning ### - After scraping data for several days, now we can remove any of the duplicates.

df = pd.DataFrame.drop_duplicates(df)

NameError: name 'df' is not defined

In [11]:
# Let's clean up the locations column.
def locations(a):
    import re
    result = re.sub(r"[0-9]", "", str(a))
    return re.sub(r'\([^)]*\)', '', result)

In [ ]:
df['Locations'] = df['Locations'].apply(locations)

### Save your results as a CSV

In [69]:
# # Let's export the dataframe into a CSV format.
# export_csv = cleaned_df.to_csv('exported_dataframe.csv', index= None, header=True)

### The Code Below Allows Us To Append To Our Existing CSV Data From Multiple Scrapes

In [50]:
df.to_csv('exported_dataframe.csv', mode='a', header=False, index=False)

NameError: name 'df' is not defined

## QUESTION 2: Factors that distinguish job category

Using the job postings you scraped for part 1 (or potentially new job postings from a second round of scraping), identify features in the data related to job postings that can distinguish job titles from each other. There are a variety of interesting ways you can frame the target variable, for example:
- What components of a job posting distinguish data scientists from other data jobs?
- What features are important for distinguishing junior vs. senior positions?
- Do the requirements for titles vary significantly with industry (e.g. healthcare vs. government)?

You may end up making multiple classification models to tackle different questions. Be sure to clearly explain your hypotheses and framing, any feature engineering, and what your target variables are. The type of classification model you choose is up to you. Be sure to interpret your results and evaluate your model's performance.

#### Load in the the data of scraped salaries

In [6]:
cleaned_df = pd.read_csv("exported_dataframe.csv")

In [108]:
# cleaned_df = cleaned_df.drop_duplicates()

#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median).

We could also perform Linear Regression (or any regression) to predict the salary value here. Instead, we are going to convert this into a _binary_ classification problem, by predicting two classes, HIGH vs LOW salary.

While performing regression may be better, performing classification may help remove some of the noise of the extreme salaries. We don't have to choose the `median` as the splitting point - we could also split on the 75th percentile or any other reasonable breaking point.

In fact, the ideal scenario may be to predict many levels of salaries.

In [14]:
# Calculating The Median.

median = cleaned_df['Salaries($)'].median()
print(median)

100000.0


In [16]:
def salary_class (x, median=median):
    if x > median:
        return 'HIGH SALARY'
    else:
        return 'LOW SALARY'

In [17]:
cleaned_df['Salary_Class'] = cleaned_df['Salaries($)'].apply(salary_class)

#### Thought experiment: What is the baseline accuracy for this model?

In [18]:
## The Baseline Accuracy Will Be The Most Frequent Class Within The Distribution

In [19]:
cleaned_df['Salary_Class'].value_counts()

LOW SALARY     273
HIGH SALARY    271
Name: Salary_Class, dtype: int64

In [20]:
# Therefore Our Baseline Accuracy Is 50.182%

(cleaned_df[(cleaned_df['Salary_Class'] == 'LOW SALARY')].shape[0] / cleaned_df.shape[0]) * 100

50.18382352941176

In [21]:
cleaned_df[cleaned_df['Ratings'].notnull()].head(15)

,Salaries($),Titles,Locations,Companies,Ratings,Salary_Class
152,140000.0,MTC Technical Architect-Data & AI,"Chicago, IL",Microsoft,87.0,HIGH SALARY
153,64650.0,Management Operations Analyst II,"Cook County, IL",State of Illinois,73.0,LOW SALARY
154,121000.0,Data Scientist,"Chicago, IL",Relativity,65.0,HIGH SALARY
155,57141.0,Research Analyst - Oakton Community College,"Des Plaines, IL",Oakton Community College,89.0,LOW SALARY
156,53000.0,Data Science Educator,"Boulder, CO",Battelle,73.0,LOW SALARY
157,63161.0,"Computer Scientist, ZP-1550-II (GS-7/10 equiva...","Boulder, CO",US Department of Commerce,87.0,LOW SALARY
158,133500.0,Data Scientist - Bioinformatics,"Denver, CO",Transamerica,71.0,HIGH SALARY
160,129500.0,Data Scientist,"Denver, CO",ICR,73.0,HIGH SALARY
161,123774.0,Chemistry Program Manager - 2040,"Denver, CO",State of Colorado Job Opportunities,70.0,HIGH SALARY
162,5000.0,UX Copywriter / Content Strategist,"Denver, CO",AETNA,72.0,LOW SALARY


In [22]:
#Feature Engineering 

#1. Creating The States.
cleaned_df['State'] = cleaned_df['Locations'].apply(lambda x: str(x).split()[-1])

In [23]:
#2. Separating Out The Location From State.
cleaned_df['Locations'] = cleaned_df['Locations'].apply(lambda x: (str(" ".join(str(x).split()[:-1])).replace(',','')))

In [24]:
df = cleaned_df

In [25]:
df

,Salaries($),Titles,Locations,Companies,Ratings,Salary_Class,State
0,120500.0,Eicoff - Data Scientist,Chicago,Ogilvy,NaN,HIGH SALARY,IL
1,115000.0,Insurance Data Scientist,Chicago,TransUnion,NaN,HIGH SALARY,IL
2,104000.0,Data Analyst / Scientist,Chicago,Centric Digital,NaN,HIGH SALARY,IL
3,142000.0,Arity-Data Scientist-Claims Analytics,Chicago,Allstate,NaN,HIGH SALARY,IL
4,64650.0,Management Operations Analyst II,Cook County,State of Illinois,NaN,LOW SALARY,IL
5,139000.0,Data Science Manager - Artificial Intelligence...,Chicago,Capgemini,NaN,HIGH SALARY,IL
6,5000.0,Lead Data Scientist – Digital Engagement,Denver,AETNA,NaN,LOW SALARY,CO
7,81028.5,"Physical Scientist, GS-1301-09/11/12 (PERM-DEU...",Lakewood,US Department of the Interior,NaN,LOW SALARY,CO
8,5000.0,UX Copywriter / Content Strategist,Denver,AETNA,NaN,LOW SALARY,CO
9,93000.0,Software Engineer,Boulder,Twitter,NaN,LOW SALARY,CO


### Create a classification model to predict High/Low salary. 


#### Model based on location:

- Start by ONLY using the location as a feature. 
- Use logistic regression with both statsmodels and sklearn.
- Use a further classifier you find suitable.
- Remember that scaling your features might be necessary.
- Display the coefficients/feature importances and write a short summary of what they mean.

#### Model taking into account job levels and categories:

- Create a few new variables in your dataframe to represent interesting features of a job title.
- For example, create a feature that represents whether 'Senior' is in the title or whether 'Manager' is in the title. 
- Incorporate other text features from the title or summary that you believe will predict the salary.
- Then build new classification models including also those features. Do they add any value? 
- Tune your models by testing parameter ranges, regularization strengths, etc. Discuss how that affects your models. 
- Discuss model coefficients or feature importances as applicable.

#### Model evaluation:

Your boss would rather tell a client incorrectly that they would get a lower salary job than tell a client incorrectly that they would get a high salary job. Adjust one of your models to ease his mind, and explain what it is doing and any tradeoffs. 


- Use cross-validation to evaluate your models. 
- Evaluate the accuracy, AUC, precision and recall of the models. 
- Plot the ROC and precision-recall curves for at least one of your models.

In [61]:
#Importing Sci-kit + Stats Models Dependencies
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, cross_val_score

#Models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import MultinomialNB, ComplementNB

#GridSearch
from sklearn.model_selection import GridSearchCV

In [43]:
#Pre-processing
from sklearn.preprocessing import StandardScaler

In [44]:
#Scoring & Evaluation Metrics
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, precision_score, f1_score

In [45]:
#Scipy Integration for Sparse Matrixes
from scipy import sparse

In [85]:
# The Machine Learning Process 

'''

1. Dummy location and create a standardized predictor matrix.
2. Run Logistic Regression within both statsmodels and sklearn.
3. Use 2 - 4 more classifiers.
4. Display the coefficients/feature importances + write a summary.

'''

'\n\n1. Dummy location and create a standardized predictor matrix.\n2. Run Logistic Regression within both statsmodels and sklearn.\n3. Use 2 - 4 more classifiers.\n4. Display the coefficients/feature importances + write a summary.\n\n'

In [88]:
X

,Salaries($),Ratings,Titles_AI and Analytics Research Software Engineer,Titles_AWS DevOps Engineer for AI Platform,Titles_Accounting Clerk I,Titles_Analytical Chemist (ICP-MS),Titles_Appraiser,Titles_Arity-Data Scientist-Claims Analytics,Titles_Artificial Intelligence Solution Architect,Titles_Assessment Design Specialist - Statistics / Data Analytics /...,...,State_GA,State_IL,State_NJ,State_NY,State_OH,State_OR,State_PA,State_TX,State_WA,State_nan
0,120500.0,NaN,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,115000.0,NaN,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,104000.0,NaN,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,142000.0,NaN,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
4,64650.0,NaN,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5,139000.0,NaN,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
6,5000.0,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,81028.5,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,5000.0,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,93000.0,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
X = cleaned_df.copy()
X = pd.get_dummies(data=X, drop_first=True)
y = X.pop('Salary_Class')

KeyError: 'Salary_Class'

In [309]:
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=1, stratify=y)

In [310]:
# X_train_sparse = sparse.csr_matrix()
# X_test_sparse = sparse.csr_matrix()

In [26]:
models = [KNeighborsClassifier(),
          LogisticRegression(solver='lbfgs', multi_class='auto'),
          DecisionTreeClassifier(),
          SVC(gamma='scale'),
          RandomForestClassifier(n_estimators=100),
          ExtraTreesClassifier(n_estimators=100)]

for model in models:
    print(model)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    print(score)
    print('---------')

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')


NameError: name 'X_train' is not defined

In [27]:
#Display the coefficients/feature importances and write a short summary of what they mean.

from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)

log_reg_model = LogisticRegressionCV(cv=skf)
log_reg_model.fit(X_train, y_train)

print('--------')
print("The Logistic Regression Accuracy Score", round(log_reg_model.score(X_test, y_test),3))
print('--------')
print("The Cross Validated Logistic Regression Accuracy Score", round(np.mean(cross_val_score(log_reg_model, X_test, y_test, cv=skf)),3))
print('--------')

### Producing The Coefficients ####

Logistic_Regression_Coefficients = log_reg_model.coef_
coefficients_matrix = pd.DataFrame(Logistic_Regression_Coefficients, columns= X_train.columns).T


NameError: name 'X_train' is not defined

In [28]:
coefficients_matrix.sort_values(by=0,ascending=False).head(15)

NameError: name 'coefficients_matrix' is not defined

In [204]:
#Plotting The Top 10 Most Positive Residuals From The Logisitic Regression Model

fig, ax = plt.subplots(figsize=(30,12))
x_values =  coefficients_matrix.sort_values(by=0, ascending=False).head(10).index
sns.barplot(x=x_values, y='Logistic Regression Coefficients', data=coefficients_matrix.sort_values(by='0', ascending=False).head(10))
plt.title('The Top 10 Most Positive Coefficients From The Logisitic Regression Model', pad=30, fontsize='25')
plt.xlabel('Coefficients', fontsize='18', labelpad=30)
plt.ylabel('Counts', fontsize='18', labelpad=30)
plt.show()

TypeError: fit_transform() missing 1 required positional argument: 'raw_documents'

In [ ]:
#Plotting The Top 10 Most Negative Residuals From The Logisitic Regression Model

fig, ax = plt.subplots(figsize=(30,12))
x_values =  coefficients_matrix.sort_values(by=0, ascending=True).head(10).index
sns.barplot(x=x_values, y='Logistic Regression Coefficients', data=coefficients_matrix.sort_values(by='0', ascending=True).head(10))
plt.title('The Top 10 Most Negative Coefficients From The Logisitic Regression Model', pad=30, fontsize='25')
plt.xlabel('Coefficients', fontsize='18', labelpad=30)
plt.ylabel('Counts', fontsize='18', labelpad=30)
plt.show()

----------------------------------------------------------------------------------------------------------------------------------------

In [29]:
df['Ratings']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
5       NaN
6       NaN
7       NaN
8       NaN
9       NaN
10      NaN
11      NaN
12      NaN
13      NaN
14      NaN
15      NaN
16      NaN
17      NaN
18      NaN
19      NaN
20      NaN
21      NaN
22      NaN
23      NaN
24      NaN
25      NaN
26      NaN
27      NaN
28      NaN
29      NaN
       ... 
514     NaN
515    87.0
516    65.0
517    72.0
518    87.0
519    86.0
520     NaN
521    87.0
522    88.0
523    89.0
524    70.0
525    88.0
526    69.0
527     NaN
528    89.0
529     NaN
530     NaN
531     NaN
532    87.0
533    87.0
534     NaN
535     NaN
536    71.0
537     NaN
538    74.0
539     NaN
540    87.0
541     NaN
542    73.0
543    68.0
Name: Ratings, Length: 544, dtype: float64

In [46]:
#Additional Feature Engineering - NLP Text Data
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()


X = df.copy()


X = X.dropna(how='any')
X.drop(columns=['Companies', 'Salaries($)'], inplace=True)

In [47]:
X = pd.get_dummies(data=X, drop_first=True, columns=['State', 'Locations'])
y = X.pop('Salary_Class')

In [49]:
X2_train, X2_test, y2_train, y2_test = train_test_split(
X, y, test_size=0.2, random_state=1, stratify=y)

X2_train_titles = vectorizer.fit_transform(X2_train['Titles'])
X2_test_titles = vectorizer.transform(X2_test['Titles'])

X2_train.drop(columns=['Titles'], inplace=True)
X2_test.drop(columns=['Titles'], inplace=True)

X_Train_Sparse = sparse.csr_matrix(X2_train.values)
X_Test_Sparse = sparse.csr_matrix(X2_test.values)

y2_train.reset_index(drop=True)
y2_test.reset_index(drop=True)

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


0     HIGH SALARY
1     HIGH SALARY
2     HIGH SALARY
3     HIGH SALARY
4     HIGH SALARY
5      LOW SALARY
6      LOW SALARY
7     HIGH SALARY
8      LOW SALARY
9     HIGH SALARY
10     LOW SALARY
11     LOW SALARY
12     LOW SALARY
13    HIGH SALARY
14     LOW SALARY
15    HIGH SALARY
16     LOW SALARY
17    HIGH SALARY
18    HIGH SALARY
19    HIGH SALARY
20    HIGH SALARY
21     LOW SALARY
22    HIGH SALARY
23     LOW SALARY
24     LOW SALARY
25     LOW SALARY
26    HIGH SALARY
27    HIGH SALARY
28    HIGH SALARY
29     LOW SALARY
30     LOW SALARY
31    HIGH SALARY
32     LOW SALARY
33    HIGH SALARY
34     LOW SALARY
35    HIGH SALARY
36     LOW SALARY
37     LOW SALARY
38     LOW SALARY
39     LOW SALARY
40    HIGH SALARY
41    HIGH SALARY
42     LOW SALARY
43    HIGH SALARY
44     LOW SALARY
45     LOW SALARY
46     LOW SALARY
47     LOW SALARY
48     LOW SALARY
49    HIGH SALARY
50    HIGH SALARY
51     LOW SALARY
52    HIGH SALARY
53     LOW SALARY
54    HIGH SALARY
55     LOW

In [50]:
X2_train = sparse.hstack((X2_train_titles, X_Train_Sparse))
X2_test = sparse.hstack((X2_test_titles, X_Test_Sparse))

In [62]:
clf = ComplementNB()

In [63]:
clf.fit(X2_train, y2_train)

ComplementNB(alpha=1.0, class_prior=None, fit_prior=True, norm=False)

In [64]:
clf.score(X2_test, y2_test)
cross_val_score(clf,X2_test, y2_test)

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


array([0.5       , 0.78947368, 0.52631579])

In [52]:
models = [KNeighborsClassifier(),
          LogisticRegression(solver='lbfgs', multi_class='auto'),
          DecisionTreeClassifier(),
          SVC(gamma='scale'),
          RandomForestClassifier(n_estimators=100),
          ExtraTreesClassifier(n_estimators=100)]

for model in models:
    print(model)
    model.fit(X2_train, y2_train)
    print(model.score(X2_train, y2_train))
    print(model.score(X2_test, y2_test))
    print(np.mean(cross_val_score(model, X2_test, y2_test, cv=5)))
    print('---------')

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')
0.7913043478260869
0.7068965517241379
0.6909090909090909
---------
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)
0.9347826086956522
0.7241379310344828


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.756060606060606
---------
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
1.0
0.7586206896551724
0.6393939393939394
---------
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
0.5304347826086957
0.5172413793103449
0.5363636363636364
---------
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            mi

In [ ]:
min_features = 5- 20 

In [ ]:
''

#Stemming
#Changing the paramatetrs min - max features
#N-grams

In [61]:
from sklearn.metrics import confusion_matrix

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

#### Bonus:

- Answer the salary discussion by using your model to explain the tradeoffs between detecting high vs low salary positions. 
- Discuss the differences and explain when you want a high-recall or a high-precision model in this scenario.
- Obtain the ROC/precision-recall curves for the different models you studied (at least the tuned model of each category) and compare.

In [ ]:
## YOUR CODE HERE

### Summarize your results in an executive summary written for a non-technical audience.
   
- Writeups should be at least 500-1000 words, defining any technical terms, explaining your approach, as well as any risks and limitations.

In [ ]:
## YOUR TEXT HERE IN MARKDOWN FORMAT 

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### BONUS

Convert your executive summary into a public blog post of at least 500 words, in which you document your approach in a tutorial for other aspiring data scientists. Link to this in your notebook.

In [ ]:
## YOUR LINK HERE IN MARKDOWN FORMAT 